In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWINB", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWINB", "T"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################


# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 4
Maxiterations_turbine = 30

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True
# Расчет для работы с теплофикацией
Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
time_ac=3600
PKM_zaryad = True
PKM_razryad=False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")



Iter_pkm = 20
pkm_pgu_tol = 10**1

start_time = time.time()
Gst = [water_streams0.at["DROSVD-ST", "G"]]
Ggpk = [water_streams0.at["SMESH-GPK", "G"]]

for i in range(Iter_pkm):
    if i < 7:
        Maxiterations_KU_TU_new = int(2)
        Maxiterations_cotel_new = int(3)
        # print(Maxiterations_KU_TU_new,Maxiterations_cotel_new)

    else:
        Maxiterations_KU_TU_new = Maxiterations_KU_TU
        Maxiterations_cotel_new = Maxiterations_KU_TU
    

    from test import calculate_CCGT
    gas_streams = calculate_CCGT(Maxiterations_KU_TU_new, Maxiterations_cotel_new, Maxiterations_turbine,
                                gas_streams0,water_streams0,
                                GTU_ISO,GTU_input,
                                gas_streams,water_streams,
                                heaters,electric,
                                Gas_turbine,gas0,
                                water,PKM_zaryad,PKM_razryad,
                                syngas_streams,Calcmethod,
                                Calctolerance,KPD_PN,KPD_KN,KPD_to,KPD_SP,
                                steamVD_fraction_to_turbine,Teplo,accumulation,time_ac)
    ####################################
    print(f"Время {i+1} итерации расчета КУ+ТУ с ПКМ:--- %s сек. --- {round((time.time() - start_time), 1)}")
    Gst.append(round(water_streams.at["PEVD-DROSVD", "G"], 2))
    Ggpk.append(round(water_streams.at["SMESH-GPK", "G"], 2))
    
    print("Gst", Gst)
    print("Ggpk", Ggpk)
    
    Err1 = abs((Gst[i] - Gst[i - 1]) / (Gst[i]) * 100)
    Err3 = abs((Ggpk[i] - Ggpk[i - 1]) / (Ggpk[i]) * 100)

    if i == Iter_pkm - 1:
        print("Достигнуто максимальнео количество итераций КУ+ПТУ+ПКМ")

    if Err1 < pkm_pgu_tol and Err3 < pkm_pgu_tol:

        print(f"Расчет КУ+ПТУ+ПКМ закончен:--- %s сек. ---{round((time.time() - start_time), 1)}")
        print(Err1, Err3)

        break


Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_G: 1.1205812336586752e-14, Error_nd_G: 0.009530605425175958, Error_vd_G: 0.0
Время 1 итерации расчета КУ+ТУ:---  18.5 сек. ---
Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_G: 0.0, Error_nd_G: 0.0006105906897450648, Error_vd_G: 0.0
Время 2 итерации расчета КУ+ТУ:---  34.9 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ 2
Pnd_it [0.56396, 0.46714]
Pvd_it [5.8763, 4.55544]
fin КУ и ТУ:--- 34.9 сек. ---
Время 1 итерации расчета КУ+ТУ с ПКМ:--- %s сек. --- 35.2
Gst [nan, 75.85]
Ggpk [78.16, 77.79]
Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество и

In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.010558,0.1,958.533980,503.497387,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,81.101140,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,581.264330,0.1,1037.969147,584.598527,0.770390,0.107595,0.038659,0.076553,0.006803
PEVD-IVD,488.458890,0.1,930.019633,584.598527,0.770390,0.107595,0.038659,0.076553,0.006803
IVD-EVD,278.638484,0.1,694.510299,584.598527,0.770390,0.107595,0.038659,0.076553,0.006803
EVD-PPND,207.293329,0.1,617.003442,584.598527,0.770390,0.107595,0.038659,0.076553,0.006803
PPND-IND,205.046645,0.1,614.581301,584.598527,0.770390,0.107595,0.038659,0.076553,0.006803
IND-GPK,155.813333,0.1,561.758164,584.598527,0.770390,0.107595,0.038659,0.076553,0.006803
GPK-out,102.234409,0.1,504.762479,584.598527,0.770390,0.107595,0.038659,0.076553,0.006803


In [3]:
water_streams

,T,P,H,G,S,X
AIR,15.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,542.751866,4.164761,3542.199942,83.326572,NaN,NaN
IVD-PEVD,266.474836,5.205886,2792.443065,83.326572,NaN,NaN
EVD-IVD,264.340792,5.205886,1156.607633,83.326572,NaN,NaN
PEN-EVD,146.073146,5.205886,618.276094,83.326572,NaN,NaN
BND-PEN,145.281374,0.418915,611.848859,83.326572,NaN,NaN
PPND-DROSND,188.395675,0.410730,2835.897973,14.645139,NaN,NaN
IND-PPND,145.281374,0.418915,2740.150515,14.645139,NaN,NaN
GPK-IND,145.281374,0.418915,617.938318,97.971711,NaN,NaN
GPK-REC,145.281374,0.418915,617.938318,97.969330,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3683.821929,73.023558,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3023.629730,73.023558,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-SGaccum,275.0,2.0,2545.703624,73.023558,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
SGaccum-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COMP-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COMB-TURB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TURB-COOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COOL-EX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
electric

,Ni,N,KPDm,KPD
PEN,0.535560,0.610464,0.877300,0.807400
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,150.750600,NaN,33.348086
DK,NaN,6.269747,NaN,NaN
Turbine,23.072977,NaN,NaN,NaN
Tots1,8.540964,NaN,NaN,0.836804
Tots2,4.128367,NaN,NaN,0.836907
Tots3,2.390176,NaN,NaN,0.814865
Tots4,8.013471,NaN,NaN,0.766665


In [6]:
heaters

,Qw,Qg,KPD
PEVD,62476.055784,63107.127054,0.99
IVD,136301.625348,137678.409443,0.99
EVD,44857.290782,45310.394729,0.99
PPND,1401.820313,1415.980114,0.99
IND,30571.524707,30880.327987,0.99
GPK,32986.397812,33319.593749,0.99
SP2,NaN,NaN,NaN
SP1,NaN,NaN,NaN
OD,NaN,NaN,NaN
Strans,214704.942931,NaN,NaN
